In [ ]:
import ee
import geemap.core as geemap
ee.Authenticate()
ee.Initialize(project='wildfire-lab')

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/content/request_data.csv", index_col=0, parse_dates=['start_date', 'end_date'])
df['start_date'] = pd.to_datetime(df['start_date']).dt.strftime('%Y-%m-%dT%H:%M:%S')
df['end_date'] = pd.to_datetime(df['end_date']).dt.strftime('%Y-%m-%dT%H:%M:%S')


# ERA 5

In [ ]:
era5 = ee.ImageCollection('ECMWF/ERA5/DAILY')

one row, one band

In [ ]:
def fetch_ee_data(start_date, end_date, lat, long, scale=1000):
    poi = ee.Geometry.Point([long, lat])
    collection = ee.ImageCollection('ECMWF/ERA5/DAILY')\
        .filter(ee.Filter.date(start_date, end_date))\
        .select('mean_2m_air_temperature')
    mean_temp = collection.mean().reduceRegion(reducer=ee.Reducer.mean(), geometry=poi, scale=scale).get('mean_2m_air_temperature')
    try:
        result = mean_temp.getInfo()
        return result
    except ee.EEException as e:
        return None
row = df.iloc[0]
result = fetch_ee_data(row['start_date'], row['end_date'], row['poo_lat'], row['poo_lon'])
print(result)

286.4913024902344


one row, different bands

In [ ]:
era5_features = ['mean_2m_air_temperature', 'minimum_2m_air_temperature', 'maximum_2m_air_temperature', 'dewpoint_2m_temperature', 'total_precipitation', 'surface_pressure', 'u_component_of_wind_10m', 'v_component_of_wind_10m']
poi = ee.Geometry.Point([row['poo_lon'], row['poo_lat']])
collection = ee.ImageCollection('ECMWF/ERA5/DAILY')\
        .filter(ee.Filter.date(row['start_date'], row['end_date']))\
        .filterBounds(poi)\
        .select(era5_features)

In [ ]:
collection

In [ ]:
def flatten_collection(collection):
    first_image = ee.Image(collection.first()).select([])

    def accumulate_bands(image, previous):
        previous = ee.Image(previous)
        image = ee.Image(image)
        date = image.date().format('YYYYMMdd')
        renamed_bands = image.bandNames().map(lambda b: ee.String(b).cat('_').cat(date))
        image_renamed = image.select(image.bandNames(), renamed_bands)
        return previous.addBands(image_renamed)

    flattened_image = ee.Image(collection.iterate(accumulate_bands, first_image))

    return flattened_image

flattened_image = flatten_collection(collection)

In [ ]:
col = collection.select('mean_2m_air_temperature').filter(ee.Filter.bounds(poi.buffer(1000)));

In [ ]:
image

In [ ]:
image = collection.first()
row = df.iloc[0]
poi = ee.Geometry.Point([row['poo_lon'], row['poo_lat']]).buffer(27.8*1000*2)
data = image.reduceRegion(
    reducer=ee.Reducer.mean(),  # Or another appropriate reducer
    geometry=poi,
    scale=27800,  # Scale in meters, ERA5's approximate spatial resolution is 0.25 degrees, or about 27.8 km
    maxPixels=1e9,
    crs="EPSG:4326"
)
data

In [ ]:
poi = ee.Geometry.Point(row['poo_lon'], row['poo_lat'])
buffered_poi = poi.buffer(25000)
bounding_box = buffered_poi.bounds()
coords = bounding_box.coordinates()
polygon = ee.Geometry.Polygon(coords)
print(polygon.getInfo())

NameError: name 'unique_corners' is not defined

In [ ]:
polygon

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "functionInvocationValue": {
          "functionName": "Geometry.coordinates",
          "arguments": {
            "geometry": {
              "functionInvocationValue": {
                "functionName": "Geometry.bounds",
                "arguments": {
                  "geometry": {
                    "functionInvocationValue": {
                      "functionName": "Geometry.buffer",
                      "arguments": {
                        "distance": {
                          "constantValue": 25000
                        },
                        "geometry": {
                          "functionInvocationValue": {
                            "functionName": "GeometryConstructors.Point",
                            "arguments": {
                              "coordinates": {
                                "constantValue": [
                                  -71.25,
                                  43.78
                                ]
                              }
                            }
                          }
                        }
                      }
                    }
                  }
                }
              }
            }
          }
        }
      }
    }
  }
})

In [ ]:
m = geemap.Map()
m.set_center(row['poo_lon'], row['poo_lat'],10)
m.add_layer(image, {'min': 250, 'max':300}, 'Landsat image')
m.add_layer(poi, {'color': 'black'})
m.add_layer(polygon, {'color': 'blue'})
display(m)


Map(center=[43.78, -71.25], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…

In [ ]:
data

In [ ]:
Object (8 properties)
dewpoint_2m_temperature:272.9349060058594
maximum_2m_air_temperature:292.76483154296875
mean_2m_air_temperature:285.95001220703125
minimum_2m_air_temperature:278.7554626464844
surface_pressure:99310.0078125
total_precipitation:0
u_component_of_wind_10m:1.9221104383468628
v_component_of_wind_10m:-0.5183467864990234

In [ ]:
col = collection.select('mean_2m_air_temperature').filter(ee.Filter.bounds(poi.buffer(1000)));

firstImage = col.first()
Map = geemap.Map()
Map.addLayer(data, {'min': 250, 'max': 300}, 'Mean Temperature');
first_image = collection.first()
Map.centerObject(poi, 10)
Map

Map(center=[43.78, -71.25], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…

In [ ]:
firstImage

In [ ]:
import numpy as np

In [ ]:
def fetch_era5_data_for_fire(fire_row, era5_features, scale=1000):
    start_date = fire_row['start_date']
    end_date = fire_row['end_date']
    poi = ee.Geometry.Point([fire_row['poo_lon'], fire_row['poo_lat']]).buffer(1000)  # Example buffer

    collection = ee.ImageCollection('ECMWF/ERA5/DAILY')\
        .filter(ee.Filter.date(start_date, end_date))\
        .select(era5_features)
    return collection

num_fires = len(df)
num_days = 20
num_features = len(era5_features)
tensor_shape = (num_fires, num_days, num_features)
data_tensor = np.zeros(tensor_shape)
fires= []
for index, row in df.iloc[:2].iterrows():
    fire_data = fetch_era5_data_for_fire(row, era5_features)
    fires.append(fire_data)

In [ ]:
fires[0]

In [ ]:
import enum
class DataType(enum.Enum):
  ELEVATION_SRTM = 1
  VEGETATION_VIIRS = 2
  DROUGHT_GRIDMET = 3
  WEATHER_ERA5 = 4
  WEATHER_GRIDMET = 5
  FIRE_MODIS = 6
  POPULATION = 7
DATA_SOURCES = {
  DataType.ELEVATION_SRTM: 'USGS/SRTMGL1_003',
  DataType.VEGETATION_VIIRS: 'NOAA/VIIRS/001/VNP13A1',
  DataType.DROUGHT_GRIDMET: 'GRIDMET/DROUGHT',
  DataType.WEATHER_ERA5: 'ECMWF/ERA5/DAILY',
  DataType.WEATHER_GRIDMET: 'IDAHO_EPSCOR/GRIDMET',
  DataType.FIRE_MODIS: 'MODIS/006/MOD14A1',
  DataType.POPULATION: 'CIESIN/GPWv411/GPW_Population_Density'
}

In [ ]:
population = ee.ImageCollection(DATA_SOURCES[DataType.POPULATION])
population = population.filterDate("2015-01-01",
                                     "2016-01-01").median().rename('population')
population.getInfo()

{'type': 'Image',
 'bands': [{'id': 'population',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

In [ ]:
image_list = [population]

In [ ]:
DEFAULT_KERNEL_SIZE=128
def convert_features_to_arrays(
    image_list,
    kernel_size = DEFAULT_KERNEL_SIZE,
):
  """Converts a list of EE images into `(kernel_size x kernel_size)` tiles.

  Args:
    image_list: The list of EE images.
    kernel_size: The size of the tiles (kernel_size x kernel_size).

  Returns:
    An EE image made of (kernel_size x kernel_size) tiles.
  """
  feature_stack = ee.Image.cat(image_list).float()
  kernel_list = ee.List.repeat(1, kernel_size)  # pytype: disable=attribute-error
  kernel_lists = ee.List.repeat(kernel_list, kernel_size)  # pytype: disable=attribute-error
  kernel = ee.Kernel.fixed(kernel_size, kernel_size, kernel_lists)
  return feature_stack.neighborhoodToArray(kernel)

In [ ]:
convert_features_to_arrays(fires[0])

/usr/local/lib/python3.10/dist-packages/eerepr/repr.py:57: UserWarning: Getting info failed with: 'Image.float, argument 'value': Invalid type.
Expected type: Image<unknown bands>.
Actual type: ImageCollection.'. Falling back to string repr.
  warn(f"Getting info failed with: '{e}'. Falling back to string repr.")


In [ ]:
feature_stack = ee.Image.cat(fires).float()

In [ ]:
feature_stack

/usr/local/lib/python3.10/dist-packages/eerepr/repr.py:57: UserWarning: Getting info failed with: 'Image.addBands, argument 'dstImg': Invalid type.
Expected type: Image<unknown bands>.
Actual type: ImageCollection.'. Falling back to string repr.
  warn(f"Getting info failed with: '{e}'. Falling back to string repr.")
